### Lipohilicity dataset

An example of regression task.

### 1. Generate descriptors

In [ ]:
# Import necessary packages
# -------------------------
import os
import sys
current_path = os.getcwd()
sys.path.append(current_path)
sys.path.append(os.path.join(sys.path[0], ".."))
from spoc.process import generate_descriptors

# Parameters
# ----------
task_name = "Lipophilicity"

# Download dataset by using the linkage
url = "https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/Lipophilicity.csv"

# Data folder
data_file = "../data/sample_data/Lipophilicity.csv"
feature_file = "./features/all_descriptor_set--Lipophilicity.pkl.zip"

# SMILES column
smiles_col = "smiles"

# If the test_mode="test", then a small amount of data will be used for test.
test_mode = "production"

# Descriptor generation
# ---------------------
# The generated descriptors will be stored with *.pkl.zip format in ./features/
generate_descriptors.load_data(
    task_name, url, data_file, feature_file, smiles_col, test_mode)

#### 2. Descriptor screening
Evaluate the performance of various descriptors by using random forest.

In [2]:
# Import necessary packages
# -------------------------
import pandas as pd
import numpy as np
import os
import sys
current_path = os.getcwd()
sys.path.append(current_path)
sys.path.append(os.path.join(sys.path[0], ".."))
from spoc.process import generate_dataset
from spoc.model import rf

import importlib
importlib.reload(rf)

# Parameters
# ----------
# Data folder
data_file = "../data/sample_data/Lipophilicity.csv"
feature_file = "./features/all_descriptor_set--Lipophilicity.pkl.zip"
output_file = "./output/rf_SF_opt--Lipophilicity.csv"

# The data column of target y
task_col = "exp"

# The data column of SMILES
smiles_col = "smiles"

# Data splitting mode, including:
# RandomSplitter, ScaffoldSplitter, SingletaskStratifiedSplitter
# The recommended splitting mode can be used by refering the published paper.
split_mode = "RandomSplitter"

# Random seed.
# In this case, 5 random seeds are generated by 5 repeated tests.
rnds = range(0, 500, 100)

# Random seed for model training.
model_rnd = 42

# Load descriptor set & dataset
# -----------------------------
data_df, desc_set_df = generate_dataset.load_dataset(data_file, feature_file)
print(f"data_df.shape: {data_df.shape}")
print(f"desc_set_df.shape: {desc_set_df.shape}")

# Model traning
# Screening the performance of all descriptors
#---------------------------------------------
# Feature list
feature_types = desc_set_df.columns
print(f"feature_types: {feature_types}")
# Random forest
# Hyper parameters of regression task.
n_estimators, criterion, max_features, max_depth = 100, 'squared_error', 'auto', 100

result = []
for i, feat_type in enumerate(feature_types):

    print(f"{'-'*30}")
    print(f"{i}th task: {feat_type}")

    mae_test_list, rmse_test_list, r2_test_list = [], [], []
    for rnd in rnds:

        # Read dataset
        X_train, X_test, y_train, y_test = generate_dataset.single_descriptor(
            data_df, desc_set_df, smiles_col, task_col, feat_type, split_mode=split_mode, frac_train=0.9, rnd=42)

        # Model training
        criteria = rf.rf_reg(X_train, X_test, y_train, y_test, n_estimators, criterion, max_features, max_depth, rnd=model_rnd)
        
        rmse_test = criteria['rmse_test']
        mae_test = criteria['mae_test']
        r2_test = criteria['r2_test']
        
        rmse_test_list.append(rmse_test)
        mae_test_list.append(mae_test)
        r2_test_list.append(r2_test)

        mae_test_ave = round(np.average(mae_test_list), 3)
        rmse_test_ave = round(np.average(rmse_test_list), 3)
        r2_test_ave = round(np.average(r2_test_list), 3)
        mae_test_std = round(np.std(mae_test_list), 3)
        rmse_test_std = round(np.std(rmse_test_list), 3)
        r2_test_std = round(np.std(r2_test_list), 3)

    result.append([feat_type, n_estimators, criterion, max_features, max_depth, str(mae_test_list), str(rmse_test_list), str(
        r2_test_list), mae_test_ave, rmse_test_ave, r2_test_ave, mae_test_std, rmse_test_std, r2_test_std])

df = pd.DataFrame(result, columns=['feat_type', 'n_estimators', 'criterion', 'max_features', 'max_depth', 'mae_test_list',
                  'rmse_test_list', 'r2_test_list', 'mae_test_ave', 'rmse_test_ave', 'r2_test_ave', 'mae_test_std', 'rmse_test_std', 'r2_test_std'])
# Save the results
df.to_csv(output_file)

data_df.shape: (4200, 3)
desc_set_df.shape: (4200, 277)
feature_types: Index(['Mordred', 'PubChem', 'MACCSKeys', 'RDKitDescriptors', 'klekota-roth',
       'Estate', 'EstateIndices', 'shortestpath_size512',
       'shortestpath_size1024', 'shortestpath_size2048',
       ...
       'FeaturedMorgan_radius6_size512', 'FeaturedMorgan_radius6_size1024',
       'FeaturedMorgan_radius6_size2048', 'FeaturedMorgan_radius6_size3072',
       'FeaturedMorgan_radius6_size4096', 'FeaturedMorgan_radius8_size512',
       'FeaturedMorgan_radius8_size1024', 'FeaturedMorgan_radius8_size2048',
       'FeaturedMorgan_radius8_size3072', 'FeaturedMorgan_radius8_size4096'],
      dtype='object', length=277)
------------------------------
0th task: Mordred
------------------------------
1th task: PubChem
------------------------------
2th task: MACCSKeys
------------------------------
3th task: RDKitDescriptors
------------------------------
4th task: klekota-roth
------------------------------
5th task: Estat

#### 3. SPOC screening
Evaluate the performance of various S+POC combination by using random forest.m

In [ ]:
# Import necessary packages
# -------------------------
import pandas as pd
import numpy as np
import os
import sys
current_path = os.getcwd()
sys.path.append(current_path)
sys.path.append(os.path.join(sys.path[0], ".."))
from spoc.process import generate_dataset
from spoc.model import rf

import importlib
importlib.reload(generate_dataset)
importlib.reload(rf)

# Parameters
# ----------
# Data folder
data_file = "../data/sample_data/Lipophilicity.csv"
feature_file = "./features/all_descriptor_set--Lipophilicity.pkl.zip"
rf_SF_opt_file = "./output/rf_SF_opt--Lipophilicity.csv"
output_file = "./output/rf_SPOC_opt--Lipophilicity.csv"

# target y value
task_col = "exp"

# SMILES column
smiles_col = "smiles"

# Data splitting mode
split_mode = "RandomSplitter"

# The criteria used for descriptor evaluation
# In this case, rmse serves as the criteria
criterion_col = "rmse_test_ave"

# Performance sort order, smaller is better for rmse, so ascending=True  
ascending = True

# Random seed
rnds = range(0, 500, 100)

# Model random seed
model_rnd = 42

# Load descriptor set & dataset
# -----------------------------
data_df, desc_set_df = generate_dataset.load_dataset(data_file, feature_file)
print(f"data_df.shape: {data_df.shape}")
print(f"desc_set_df.shape: {desc_set_df.shape}")

# 20 best S+POC combination
# --------------------------
# exclude ['Mordred','RDKitDescriptors']
df = df[~df['feat_type'].isin(['Mordred','RDKitDescriptors'])]

# RMSE: smaller is better
df = df.sort_values(by=[criterion_col], axis=0, ascending=ascending)

# Choose the best 20 fingerprint 
df = df.iloc[:20,:]
feature_type_Ss = df['feat_type'].values
print(f"20 best feature_Ss: {feature_type_Ss}")
feature_type_POCs = ['Mordred','RDKitDescriptors']
print(f"feature_POCs: {feature_type_POCs}")

# Model traning
# Screening the performance of all descriptors
#---------------------------------------------
# feature list
feature_types = desc_set_df.columns
print(f"feature_types: {feature_types}")

# Random forest
# Hyper parameters of regression
n_estimators, criterion, max_features, max_depth = 100, 'squared_error', 'auto', 100

result = []
for i, feat_type_S in enumerate(feature_type_Ss):
    for j, feat_type_POC in enumerate(feature_type_POCs):
        print(f"{'-'*30}")
        print(f"{i}th task: {feat_type}")

        mae_test_list, rmse_test_list, r2_test_list = [], [], []
        for rnd in rnds:
            
            # Load dataset
            X_train, X_test, y_train, y_test = generate_dataset.SPOC_descriptor(data_df, desc_set_df, smiles_col, task_col, feat_type_S, feat_type_POC, split_mode, frac_train=0.9, rnd=42)

            # Model training
            criteria = rf.rf_reg(
                X_train, X_test, y_train, y_test, n_estimators, criterion, max_features, max_depth, rnd=model_rnd)
            
            rmse_test = criteria['rmse_test']
            mae_test = criteria['mae_test']
            r2_test = criteria['r2_test']
                   
            rmse_test_list.append(rmse_test)
            mae_test_list.append(mae_test)
            r2_test_list.append(r2_test)

        mae_test_ave = round(np.average(mae_test_list), 3)
        rmse_test_ave = round(np.average(rmse_test_list), 3)
        r2_test_ave = round(np.average(r2_test_list), 3)
        mae_test_std = round(np.std(mae_test_list), 3)
        rmse_test_std = round(np.std(rmse_test_list), 3)
        r2_test_std = round(np.std(r2_test_list), 3)

        result.append([feat_type_S, feat_type_POC, n_estimators, criterion, max_features, max_depth, str(mae_test_list), str(rmse_test_list), str(
            r2_test_list), mae_test_ave, rmse_test_ave, r2_test_ave, mae_test_std, rmse_test_std, r2_test_std])

print(result)
df = pd.DataFrame(result, columns=['feat_type_S', 'feat_type_POC', 'n_estimators', 'criterion', 'max_features', 'max_depth', 'mae_test_list', 'rmse_test_list', 'r2_test_list', 'mae_test_ave', 'rmse_test_ave', 'r2_test_ave', 'mae_test_std', 'rmse_test_std', 'r2_test_std'])

# Save the results
df.to_csv(output_file)


#### 4. Bayes Optimization by using LightGBM 

In [ ]:
# Import necessary packages
# -------------------------
import pandas as pd
import numpy as np
import os
import sys
current_path = os.getcwd()
sys.path.append(current_path)
sys.path.append(os.path.join(sys.path[0], ".."))
from spoc.model import lightgbm
from spoc.process import generate_dataset

import importlib
importlib.reload(generate_dataset)
importlib.reload(lightgbm)

# Parameters
# ----------
# Data folder
data_file = "../data/sample_data/Lipophilicity.csv"
feature_file = "./features/all_descriptor_set--Lipophilicity.pkl.zip"
rf_SPOC_opt_file = "./output/rf_SPOC_opt--Lipophilicity.csv"
output_file = "./output/lgb_SPOC_opt--Lipophilicity.csv"

# target y value
task_col = "exp"

# SMILES column
smiles_col = "smiles"

# Data splitting mode
split_mode = "RandomSplitter"

# The criteria used for descriptor evaluation
# In this case, rmse serves as the criteria
criterion_col = "rmse_test_ave"

# Performance sort order, smaller is better for rmse, so ascending=True 
ascending = True

# Random seed
rnds = range(0, 500, 100)

# Model random seed
model_rnd = 42

# Task type
task_type = "regression"

# Bayes Optimization
# Initial iteration, more is better
init_iter = 3

# Optimization times, more is better
n_iters = 5

# LightGBM score function
feval, ascending = lightgbm.feval_value(criterion_col)

# Load descriptor set & dataset
# -----------------------------
data_df, desc_set_df = generate_dataset.load_dataset(data_file, feature_file)
data_df = data_df.iloc[0:100, :]
print(f"data_df.shape: {data_df.shape}")
print(f"desc_set_df.shape: {desc_set_df.shape}")

# Best S+POC combination
# --------------------------
df = pd.read_csv(rf_SPOC_opt_file)
df = df.sort_values(by=[criterion_col], axis=0, ascending=ascending)
feat_type_S = df['feat_type_S'].values[0]
feat_type_POC = df['feat_type_POC'].values[0]
print(f"Best S+POCs combination: {feat_type_S} + {feat_type_POC}")

# Start hyper opt:
#------------------------------------------
mae_test_list, rmse_test_list, r2_test_list = [], [], []
result = []
for seed in rnds:
    # Load data
    X_train, X_test, y_train, y_test = generate_dataset.SPOC_descriptor(data_df, desc_set_df, smiles_col, task_col, feat_type_S, feat_type_POC, split_mode, frac_train=0.9, rnd=42)

    print(f"X_train.shape: {X_train.shape}, X_test.shape: {X_test.shape}, y_train.shape: {y_train.shape}, y_test.shape: {y_test.shape}")

    # Optimization
    best_params = lightgbm.bayesopt_lgb(X_train, y_train, init_iter, n_iters, feval, criterion_col, pds='default', random_state=model_rnd, seed=seed, task=task_type)

    # Model training
    criteria = lightgbm.lgb_train(X_train, X_test, y_train, y_test, best_params, feval, seed=seed, task="regression")
    rmse_test = criteria['rmse_test']
    mae_test = criteria['rmse_test']
    r2_test = criteria['r2_test']
    
    mae_test_list.append(mae_test)
    rmse_test_list.append(rmse_test)
    r2_test_list.append(r2_test)

    temp_result = [seed, mae_test, rmse_test, r2_test, best_params]
    result.append(temp_result)

# Summary
mae_test_ave = np.average(mae_test_list)
rmse_test_ave = np.average(rmse_test_list)
r2_test_ave = np.average(r2_test_list)
mae_test_std = np.std(mae_test_list)
rmse_test_std = np.std(rmse_test_list)
r2_test_std = np.std(r2_test_list)

result.append(["ave", mae_test_ave, rmse_test_ave, r2_test_ave, "--"])
result.append(["std", mae_test_std, rmse_test_std, r2_test_std, "--"])

# Save results
# --------------
df = pd.DataFrame(result, columns=['entry', 'mae_test_ave', 'rmse_test_ave', 'r2_test_ave', 'best-params'])
df.to_csv(output_file)


#### 5. Bayes Optimization with XGBoost 

In [ ]:
# Import necessary packages
# -------------------------
import pandas as pd
import numpy as np
import os
import sys
current_path = os.getcwd()
sys.path.append(current_path)
sys.path.append(os.path.join(sys.path[0], ".."))
from spoc.model import xgboost
from spoc.process import generate_dataset

import importlib
importlib.reload(xgboost)
importlib.reload(generate_dataset)

# Parameters
# ----------
# Data folder
data_file = "../data/sample_data/Lipophilicity.csv"
feature_file = "./features/all_descriptor_set--Lipophilicity.pkl.zip"
rf_SPOC_opt_file = "./output/rf_SPOC_opt--Lipophilicity.csv"
output_file = "./output/xgb_SPOC_opt--Lipophilicity.csv"

# target y value
task_col = "exp"

# SMILES column
smiles_col = "smiles"

# Data splitting mode
split_mode = "RandomSplitter"

# The criteria used for descriptor evaluation
# In this case, rmse serves as the criteria
criterion_col = "rmse_test_ave"

# Performance sort order, smaller is better for rmse, so ascending=True 
ascending = True

# Random seed
rnds = range(0, 500, 100)

# Bayes Optimization
# Initial iteration, more is better
init_iter = 3

# Optimization times, more is better
n_iters = 5

# Task type
task_type = "regression"

# Model random seed
model_rnd = 42

# load descriptor set & dataset
# -----------------------------
data_df, desc_set_df = generate_dataset.load_dataset(data_file, feature_file)
print(f"data_df.shape: {data_df.shape}")
print(f"desc_set_df.shape: {desc_set_df.shape}")

# Best S+POC combination
# --------------------------
df = pd.read_csv(rf_SPOC_opt_file)
df = df.sort_values(by=[criterion_col], axis=0, ascending=ascending)
feat_type_S = df['feat_type_S'].values[0]
feat_type_POC = df['feat_type_POC'].values[0]
print(f"Best S+POCs combination: {feat_type_S} + {feat_type_POC}")

# XGBoost score function
feval, ascending = xgboost.feval_value(criterion_col)

# Start hyper opt:
#------------------------------------------
mae_test_list, rmse_test_list, r2_test_list = [], [], []
result = []
for seed in rnds:
    # Load data
    X_train, X_test, y_train, y_test = generate_dataset.SPOC_descriptor(data_df, desc_set_df, smiles_col, task_col, feat_type_S, feat_type_POC, split_mode, frac_train=0.9, rnd=42)

    print(f"X_train.shape: {X_train.shape}, X_test.shape: {X_test.shape}, y_train.shape: {y_train.shape}, y_test.shape: {y_test.shape}")

    # Optimization
    best_params = xgboost.bayesion_opt_xgb(X_train, y_train, task_type, init_iter, n_iters, feval, criterion_col, pds='default', random_state=model_rnd, seed=seed)
    
    # Model training
    criteria = xgboost.xgb_train(X_train, X_test, y_train, y_test, task_type, best_params, feval)
    rmse_test = criteria['rmse_test']
    mae_test = criteria['rmse_test']
    r2_test = criteria['r2_test']
    
    mae_test_list.append(mae_test)
    rmse_test_list.append(rmse_test)
    r2_test_list.append(r2_test)

    temp_result = [seed, mae_test, rmse_test, r2_test, best_params]
    result.append(temp_result)

# Summary
mae_test_ave = np.average(mae_test_list)
rmse_test_ave = np.average(rmse_test_list)
r2_test_ave = np.average(r2_test_list)
mae_test_std = np.std(mae_test_list)
rmse_test_std = np.std(rmse_test_list)
r2_test_std = np.std(r2_test_list)

result.append(["ave", mae_test_ave, rmse_test_ave, r2_test_ave, "--"])
result.append(["std", mae_test_std, rmse_test_std, r2_test_std, "--"])

# Save results
# --------------
df = pd.DataFrame(result, columns=['entry', 'mae_test_ave', 'rmse_test_ave', 'r2_test_ave', 'best-params'])
df.to_csv(output_file)